In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab

num_episodes = 30001

agentRadius = 10

# get size of state and action from environment

boundaryPos = [400, 590]
boundaryLength = [70,70]
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [boundaryPos[0] + boundaryLength[0], boundaryPos[1]]

moveObstacles = True
action_size = 9
obsNumber = 1
state_size = obsNumber * 2 + 1
# state_size = obsNumber + 1

Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model = True
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        if self.load_model:
            self.actor.load_weights("./Practice004_DataSave/Actor.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < boundaryPos[0]):
        xPos = boundaryPos[0]
        flagWall = -1
    elif(xPos > boundaryPos[0] + boundaryLength[0]):
        xPos = boundaryPos[0] + boundaryLength[0]
        flagWall = -1
    if(yPos < boundaryPos[1]):
        yPos = boundaryPos[1]
        flagWall = -1
    elif(yPos > boundaryPos[1] + boundaryLength[1]):
        yPos = boundaryPos[1] + boundaryLength[1]
        flagWall = -1
        
    return [xPos, yPos, flagWall]

In [4]:
def ckInit(initLoc, obsLoc):
#     if math.sqrt((initLoc[0] - obsLoc[0])**2 + (initLoc[1] - obsLoc[1])**2) <= 2*agentRadius + 1:
#         if initLoc[0] - obsLoc[0] < 0:
#             obsLoc[0] += 1
#         else:
#             obsLoc[0] -= 1
#         if initLoc[1] - obsLoc[1] < 0:
#             obsLoc[1] += 1
#         else:
#             obsLoc[1] -= 1
    # Avoid interrupting goal position
    if math.sqrt((goalPos[0] - obsLoc[0])**2 + (goalPos[1] - obsLoc[1])**2) <= 2*agentRadius + 1:
        obsLoc[0] -= 1
        obsLoc[1] += 1
    return obsLoc

In [5]:
def stateGenerator(obsPosition, agtPosition):
    returnSum = []
    for i in range(0,obsNumber):
#         returnSum = returnSum + [math.sqrt((agtPosition[0] - obsPosition[i][0])**2 + (agtPosition[1] - obsPosition[i][1])**2)]
        returnSum = returnSum + [agtPosition[0] - obsPosition[i][0], agtPosition[1] - obsPosition[i][1]]
#     returnSum = returnSum + [agtPosition[0] - 640, agtPosition[1] - 450]
    returnSum = returnSum + [math.sqrt((agtPosition[0] - goalPos[0])**2 + (agtPosition[1] - goalPos[1])**2)]
    returnSum = np.reshape(returnSum, [1, state_size])
    return returnSum

In [6]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [7]:
pygame.init()
screen = pygame.display.set_mode([1280,960])
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
    while True:
        obstaclePos[i][0] = boundaryPos[0] + random.randrange(1, boundaryLength[0])
        obstaclePos[i][1] = boundaryPos[1] + random.randrange(1, boundaryLength[1])
        if obstaclePos[i][0] <= goalPos[0] - agentRadius or obstaclePos[i][0] >= goalPos[0] + agentRadius:
            if obstaclePos[i][1] >= goalPos[1] + agentRadius or obstaclePos[i][1] <= goalPos[1] - agentRadius:
                if obstaclePos[i][0] <= initPosAgent[0] - agentRadius or obstaclePos[i][0] >= initPosAgent[0] + agentRadius:
                    if obstaclePos[i][1] >= initPosAgent[1] + agentRadius or obstaclePos[i][1] <= initPosAgent[1] - agentRadius:
                        break

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]#boundaryPos[0]
    y = initPosAgent[1]#boundaryPos[1]
    print("Episode ", e, "Starts!")
    state = stateGenerator(obstaclePos, [x,y])
    #state = np.reshape(state, [1, state_size])
    
    while not done:
        
        action = agent.get_action(state)
        
        xMove = 0
        yMove = 0
        
        [xMove, yMove] = takeAction(action)
        
        x = x + xMove
        y = y + yMove
        
           
        wallFlag = 0
        collisionFlag = 0
        [x, y, wallFlag] = ckWall(x,y)
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
#         if wallFlag == -1:
#             print("Wall!", action)
        next_state = stateGenerator(obstaclePos, [x,y])

        if(math.sqrt((x - goalPos[0])**2 + (y - goalPos[1])**2) <= 20):
            print("Goal Reached!")           
            collisionFlag = 1
            done = 1
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
                [obstaclePos[i][0], obstaclePos[i][1], _] = ckWall(obstaclePos[i][0], obstaclePos[i][1])
                obstaclePos[i] = ckInit(initPosAgent ,obstaclePos[i])

            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], 10, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= 20:
                print("Collision!")
                collisionFlag = -1
                ObjectIndex = i
                done = True 
#         if wallFlag == -1:
#             done = True
        
        if not done:
            reward = -0.1
            if wallFlag == -1:
                reward = -10
        else:
            if collisionFlag == 1:
                reward = 10000
                rList.append(1)
            elif collisionFlag == -1:
                reward = -10000
                rList.append(0)
#             next_state, reward, done, ininitPosAgentfo = env.step(action)
#             next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
#             reward = reward if not done or score == 499 else -100
        
        agent.train_model(state, action, reward, next_state, done)
        
        score += reward
        state = next_state

        if done:
            # every episode, plot the play time
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
                while True:
                    #임시로 더 좋은 학습을 위해 random range를 살짝 수정함.
                    obstaclePos[i][0] = boundaryPos[0] + random.randrange(1, boundaryLength[0])
                    obstaclePos[i][1] = boundaryPos[1] + random.randrange(1, boundaryLength[1])
                    if obstaclePos[i][0] < goalPos[0] - agentRadius or obstaclePos[i][0] > goalPos[0] + agentRadius:
                        if obstaclePos[i][1] > goalPos[1] + agentRadius or obstaclePos[i][1] < goalPos[1] - agentRadius:
                            if obstaclePos[i][0] < initPosAgent[0] - agentRadius - 1 or obstaclePos[i][0] > initPosAgent[0] + agentRadius + 1:
                                if obstaclePos[i][1] > initPosAgent[1] + agentRadius + 1 or obstaclePos[i][1] < initPosAgent[1] - agentRadius - 1:
                                    break
            episodes.append(e)
            pylab.plot(episodes, rList, 'b')
            
        #circle(Surface, color, pos, radius, width=0)
        pygame.draw.circle(screen, [100,255,100], [goalPos[0],goalPos[1]], 10, 2)
        #rect(Surface, color, Rect, width=0)
        pygame.draw.rect(screen, [255,100,100],[boundaryPos[0] - agentRadius, boundaryPos[1] - agentRadius, boundaryLength[0] + agentRadius * 2, boundaryLength[1] + agentRadius * 2],2)
        pygame.display.flip()
        screen.fill([200,200,200])
    print score
    # save the model
    if e % 50 == 0:
        agent.actor.save_weights("./Practice004_DataSave/Actor.h5")
        agent.critic.save_weights("./Practice004_DataSave/Critic.h5")
#         pylab.savefig("./Practice004_DataSave/ActorCriticGraph.png")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1161      
Total params: 1,673
Trainable params: 1,673
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')
Goal Reached!
8836.5
('Episode ', 1, 'Starts!')
G

Goal Reached!
9998.0
('Episode ', 144, 'Starts!')
Goal Reached!
9998.0
('Episode ', 145, 'Starts!')
Goal Reached!
9998.0
('Episode ', 146, 'Starts!')
Goal Reached!
9998.0
('Episode ', 147, 'Starts!')
Goal Reached!
9998.0
('Episode ', 148, 'Starts!')
Goal Reached!
9998.0
('Episode ', 149, 'Starts!')
Goal Reached!
9998.0
('Episode ', 150, 'Starts!')
Goal Reached!
9176.5
('Episode ', 151, 'Starts!')
Goal Reached!
9998.0
('Episode ', 152, 'Starts!')
Goal Reached!
9998.0
('Episode ', 153, 'Starts!')
Goal Reached!
9998.0
('Episode ', 154, 'Starts!')
Goal Reached!
9998.0
('Episode ', 155, 'Starts!')
Goal Reached!
9998.0
('Episode ', 156, 'Starts!')
Goal Reached!
9998.0
('Episode ', 157, 'Starts!')
Goal Reached!
9998.0
('Episode ', 158, 'Starts!')
Goal Reached!
9998.0
('Episode ', 159, 'Starts!')
Goal Reached!
9998.0
('Episode ', 160, 'Starts!')
Collision!
-10473.5
('Episode ', 161, 'Starts!')
Goal Reached!
9998.0
('Episode ', 162, 'Starts!')
Goal Reached!
9998.0
('Episode ', 163, 'Starts!')
G

Goal Reached!
9998.0
('Episode ', 309, 'Starts!')
Goal Reached!
9998.0
('Episode ', 310, 'Starts!')
Goal Reached!
9998.0
('Episode ', 311, 'Starts!')
Goal Reached!
9998.0
('Episode ', 312, 'Starts!')
Goal Reached!
9998.0
('Episode ', 313, 'Starts!')
Goal Reached!
9998.0
('Episode ', 314, 'Starts!')
Goal Reached!
9998.0
('Episode ', 315, 'Starts!')
Goal Reached!
9998.0
('Episode ', 316, 'Starts!')
Goal Reached!
9998.0
('Episode ', 317, 'Starts!')
Collision!
-10273.5
('Episode ', 318, 'Starts!')
Goal Reached!
8766.4
('Episode ', 319, 'Starts!')
Goal Reached!
9997.9
('Episode ', 320, 'Starts!')
Goal Reached!
9998.0
('Episode ', 321, 'Starts!')
Goal Reached!
9998.0
('Episode ', 322, 'Starts!')
Goal Reached!
9998.0
('Episode ', 323, 'Starts!')
Goal Reached!
9998.0
('Episode ', 324, 'Starts!')
Goal Reached!
9998.0
('Episode ', 325, 'Starts!')
Goal Reached!
9998.0
('Episode ', 326, 'Starts!')
Goal Reached!
9998.0
('Episode ', 327, 'Starts!')
Goal Reached!
9998.0
('Episode ', 328, 'Starts!')
G

Goal Reached!
9998.0
('Episode ', 475, 'Starts!')
Goal Reached!
9998.0
('Episode ', 476, 'Starts!')
Goal Reached!
9998.0
('Episode ', 477, 'Starts!')
Goal Reached!
9998.0
('Episode ', 478, 'Starts!')
Goal Reached!
9466.5
('Episode ', 479, 'Starts!')
Goal Reached!
9998.0
('Episode ', 480, 'Starts!')
Goal Reached!
9998.0
('Episode ', 481, 'Starts!')
Collision!
-10627.4
('Episode ', 482, 'Starts!')
Goal Reached!
9998.0
('Episode ', 483, 'Starts!')
Goal Reached!
9997.6
('Episode ', 484, 'Starts!')
Goal Reached!
Collision!
-10553.7
('Episode ', 485, 'Starts!')
Goal Reached!
9998.0
('Episode ', 486, 'Starts!')
Goal Reached!
9998.0
('Episode ', 487, 'Starts!')
Goal Reached!
9997.9
('Episode ', 488, 'Starts!')
Goal Reached!
9526.5
('Episode ', 489, 'Starts!')
Collision!
-10000.2
('Episode ', 490, 'Starts!')
Collision!
-10313.9
('Episode ', 491, 'Starts!')
Goal Reached!
9998.0
('Episode ', 492, 'Starts!')
Goal Reached!
9998.0
('Episode ', 493, 'Starts!')
Goal Reached!
9998.0
('Episode ', 494, '

Goal Reached!
9998.0
('Episode ', 640, 'Starts!')
Goal Reached!
9998.0
('Episode ', 641, 'Starts!')
Collision!
-10000.1
('Episode ', 642, 'Starts!')
Goal Reached!
9998.0
('Episode ', 643, 'Starts!')
Goal Reached!
9997.2
('Episode ', 644, 'Starts!')
Goal Reached!
9998.0
('Episode ', 645, 'Starts!')
Goal Reached!
9998.0
('Episode ', 646, 'Starts!')
Goal Reached!
9998.0
('Episode ', 647, 'Starts!')
Goal Reached!
9997.6
('Episode ', 648, 'Starts!')
Goal Reached!
9998.0
('Episode ', 649, 'Starts!')
Goal Reached!
9998.0
('Episode ', 650, 'Starts!')
Goal Reached!
9998.0
('Episode ', 651, 'Starts!')
Goal Reached!
9998.0
('Episode ', 652, 'Starts!')
Goal Reached!
9998.0
('Episode ', 653, 'Starts!')
Goal Reached!
9998.0
('Episode ', 654, 'Starts!')
Goal Reached!
9998.0
('Episode ', 655, 'Starts!')
Goal Reached!
9998.0
('Episode ', 656, 'Starts!')
Goal Reached!
9998.0
('Episode ', 657, 'Starts!')
Goal Reached!
9998.0
('Episode ', 658, 'Starts!')
Collision!
-10000
('Episode ', 659, 'Starts!')
Goal

...

Goal Reached!
9998.0
('Episode ', 29242, 'Starts!')
Collision!
-10000
('Episode ', 29243, 'Starts!')
Goal Reached!
9997.9
('Episode ', 29244, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29245, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29246, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29247, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29248, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29249, 'Starts!')
Goal Reached!
9586.5
('Episode ', 29250, 'Starts!')
Collision!
-10723.5
('Episode ', 29251, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29252, 'Starts!')
Goal Reached!
8196.5
('Episode ', 29253, 'Starts!')
Goal Reached!
9346.3
('Episode ', 29254, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29255, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29256, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29257, 'Starts!')
Collision!
-13585.1
('Episode ', 29258, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29259, 'Starts!')
Goal Reached!
9997.4
('Episode ', 29260, 'Starts!')
Goal Reached!
999

9998.0
('Episode ', 29400, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29401, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29402, 'Starts!')
Goal Reached!
9997.4
('Episode ', 29403, 'Starts!')
Goal Reached!
9946.5
('Episode ', 29404, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29405, 'Starts!')
Goal Reached!
9997.1
('Episode ', 29406, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29407, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29408, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29409, 'Starts!')
Goal Reached!
9997.2
('Episode ', 29410, 'Starts!')
Collision!
-10000
('Episode ', 29411, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29412, 'Starts!')
Goal Reached!
9997.3
('Episode ', 29413, 'Starts!')
Collision!
-10000
('Episode ', 29414, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29415, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29416, 'Starts!')
Goal Reached!
9436.5
('Episode ', 29417, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29418, 'Starts!')
Goal Reached!
9096.5
('Episode '

Goal Reached!
9998.0
('Episode ', 29559, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29560, 'Starts!')
Goal Reached!
9997.9
('Episode ', 29561, 'Starts!')
Collision!
-10223.6
('Episode ', 29562, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29563, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29564, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29565, 'Starts!')
Goal Reached!
9997.1
('Episode ', 29566, 'Starts!')
Goal Reached!
9997.9
('Episode ', 29567, 'Starts!')
Goal Reached!
9997.8
('Episode ', 29568, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29569, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29570, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29571, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29572, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29573, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29574, 'Starts!')
Goal Reached!
9606.5
('Episode ', 29575, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29576, 'Starts!')
Goal Reached!
9997.8
('Episode ', 29577, 'Starts!')
Goal Reached!

9998.0
('Episode ', 29717, 'Starts!')
Goal Reached!
9586.5
('Episode ', 29718, 'Starts!')
Goal Reached!
9566.3
('Episode ', 29719, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29720, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29721, 'Starts!')
Collision!
-10000
('Episode ', 29722, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29723, 'Starts!')
Goal Reached!
9997.7
('Episode ', 29724, 'Starts!')
Goal Reached!
9997.8
('Episode ', 29725, 'Starts!')
Goal Reached!
9997.6
('Episode ', 29726, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29727, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29728, 'Starts!')
Goal Reached!
9997.1
('Episode ', 29729, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29730, 'Starts!')
Goal Reached!
8566.5
('Episode ', 29731, 'Starts!')
Goal Reached!
9996.7
('Episode ', 29732, 'Starts!')
Collision!
-10000
('Episode ', 29733, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29734, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29735, 'Starts!')
Goal Reached!
9997.7
('Episode '

9998.0
('Episode ', 29875, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29876, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29877, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29878, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29879, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29880, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29881, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29882, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29883, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29884, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29885, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29886, 'Starts!')
Goal Reached!
9997.6
('Episode ', 29887, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29888, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29889, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29890, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29891, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29892, 'Starts!')
Goal Reached!
9998.0
('Episode ', 29893, 'Starts!')
Goal Reached!
9998.0
('Epi

In [ ]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

Percent of successful episodes: 92.4802506583%
